# Introduccion a Gradio

In [4]:
# imports

import os
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from typing import List
from langchain_ollama.llms import OllamaLLM

In [18]:
import gradio as gr

In [4]:
system_message = "You are a helpful assistant"

In [31]:
def message_ollama(prompt, model_name):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    return OllamaLLM(model=model_name).invoke(messages)

In [ ]:
prompt = "What is Gradio?"
qwen_05b = "qwen2.5-coder:0.5b"
message_ollama(prompt, qwen_05b)

## Ejemplo de User Interface

In [8]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Añadir share=True significa que puede ser accedido públicamente
# NOTA: Algunos antivírus y firewall corporativos pueden no tolerar usar share=True. Si estás en un entorno de trabajo o en una red de trabajo, sugiero desactivar esta prueba.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Agregar inbrowser=True abre una ventana de navegador automaticamente

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forzar 'Dark Mode'

Gradio se muestra en modo claro o oscuro dependiendo del conjunto de configuraciones del navegador y la
computadora. Hay una forma de forzar Gradio a mostrar el modo oscuro, pero Gradio recomienda no hacer esto ya que
debería ser una preferencia de accesibilidad ( especialmente para los usuarios). Sin embargo, para hacerlo siga estos pasos:

In [ ]:
# Define esta variable y luego pasa "force_dark_mode" al crear la interfaz.

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Inputs y Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Cambiar la funcion de demo por la que hace el llamado a nuestro LLM
model_value = gr.Text(label="LLM Model", value=qwen_05b)
view = gr.Interface(
    fn=message_ollama,
    inputs=[gr.Textbox(label="Your message:", lines=6), model_value],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Cambiemos un poco el estilo de respuesta para obtenerlo como Markdown

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_ollama,
    inputs=[gr.Textbox(label="Your message:"), model_value],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [20]:
# Ahora modifiquemos el llamado al LLM para que devuela la respuesta en modo de 'stream'
def stream_ollama(prompt, model_name):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    result = ""
    for chunk in OllamaLLM(model=model_name).stream(messages):
        result += chunk or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_ollama,
    inputs=[gr.Textbox(label="Your message:"), model_value],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [51]:
def stream_multi_model(prompt, model):
    if model=="qwen2.5-coder:0.5b":
        result = stream_ollama(prompt, "qwen2.5-coder:0.5b")
    elif model=="llama3.2":
        result = stream_ollama(prompt, "llama3.2")
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_multi_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["qwen2.5-coder:0.5b", "llama3.2"], label="Select model", value="qwen2.5-coder:0.5b")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# Building a company brochure generator

Now you know how - it's simple!

In [12]:
# A class to represent a Webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Company Name"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        # Creación de una lista que contiene todos los links, utilizando find_all para obtener todas las equitetas<a href=""></a> de html
        links = [link.get('href') for link in soup.find_all('a')]
        # asignando dicha lista al atributo links de cada instancia de clase, solo incluyendo aquellos que empiecen con `/` o `https`
        links = [link for link in links if link and (link.startswith('/') or link.startswith('https'))]
        for i, elem in enumerate(links):
            if elem.startswith('/'):
                links[i] = url + elem
        # print(f"de {len(links)} enlaces")
        # Filtrado de links relevantes
        # Definimos las palabras de interes
        common_keywords = ['company', 'about', 'contact', 'support', 'team', 'careers']
        # Calcular un puntaje de relevancia basado en el número de palabras clave comunes encontradas en la ruta del URL"
        scores = {link: sum(1 for keyword in common_keywords if re.search(r'\b' + keyword + r'\b', urlparse(link).path)) for link in links}
        # Ordenar los enlaces según su puntaje de relevancia y filtrar los mas relevantes hasta alcanzar un determinado número de enlaces
        num_relevant_links = int(len(links) * 0.2)  # Porcentaje de enlaces totales a mantener = 20%
        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        filtered_links = [link for link, score in sorted_scores[:num_relevant_links]]
        # print(f"a {len(filtered_links)} enlaces")
        self.links = filtered_links


    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

    def get_all_details(self):
        result = "Landing page:\n"
        result += self.get_contents()
        for link in self.links:
            result += f"{link}\n"
            result += Website(link).get_contents()
        return result

In [ ]:
langchain = Website('https://www.langchain.com')
print(langchain.get_all_details())

In [30]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(website):
    user_prompt = f"You are looking at a company called: {website.title}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += website.get_all_details()
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def messages_for_LLM(system_prompt, website):
    return [
        {"role": "system", "content": system_prompt}, #configuracion del system prompt
        {"role": "user", "content": get_brochure_user_prompt(website)} #configuracion del input de usuario con los datos de la pagina web
    ]

In [31]:
# Nueva implementacion para tener los prompts definidos en el bloque anterior
def stream_ollama_for_multi_model(prompt, model_name):
    result = ""
    for chunk in OllamaLLM(model=model_name).stream(prompt):
        result += chunk or ""
        yield result

def stream_multi_model(url, model):
    website = Website(url)
    prompt = messages_for_LLM(system_prompt, website)
    if model=="qwen2.5-coder:0.5b":
        result = stream_ollama_for_multi_model(prompt, "qwen2.5-coder:0.5b")
    elif model=="llama3.2":
        result = stream_ollama_for_multi_model(prompt, "llama3.2")
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_multi_model,
    inputs=[
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["qwen2.5-coder:0.5b", "llama3.2"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()